In [1]:
from pathlib import Path
import pandas as pd

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[-1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip'), WindowsPath('data/2023-04-30_13-43-lenta.zip'), WindowsPath('data/2023-05-07_21-14-lenta.zip')]
c:\works\work_002\data\2023-05-07_21-14-lenta.zip


In [3]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30635 entries, 0 to 30634
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      30635 non-null  object 
 1   Название       30635 non-null  object 
 2   Страна         30534 non-null  object 
 3   Скидка         16463 non-null  object 
 4   Цена           30635 non-null  float64
 5   Цена по карте  30633 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.4+ MB


In [4]:
mask = df['Название'].str.contains('Килька')
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower()#.contains('лента')
mask3 = mask3.str.contains('лента')
test = kilka[mask2 & mask3].sort_values(by='Цена').copy()
test.reset_index()

,index,Категория,Название,Страна,Скидка,Цена,Цена по карте
0,6087,Бакалея,"Килька ЛЕНТА в томатном соусе, 240г","Россия, 240 г",-31%,94.79,64.99


In [5]:
mask = df['Название'].str.lower().str.contains(r'(?=молоко)(?=.* пастериз)(?=.*без змж)', regex=True)
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower().str.contains('лента')
# test = kilka[mask2].sort_values(by='Цена').copy()
test = kilka.copy()
test['volume'] = test['Страна'].str.extract('(\d+)').astype(int)
test['Пр. цена'] = round((test['Цена по карте'] / (test['volume'] / 1000)), 2)
mask4 = test['volume']>600
test_min_mean = test[mask4]['Пр. цена'].nsmallest(5).mean()
test_median = test[mask4]['Пр. цена'].median()
test_max_mean = test[mask4]['Пр. цена'].nlargest(5).mean()
print(f'минимум \t{test_min_mean:.2f}')
print(f'медиана \t{test_median:.2f}')
print(f'максимум \t{test_max_mean:.2f}')
print(f'количество позиций: {test.shape[0]}')
display(test.nsmallest(5, columns='Пр. цена'))

минимум 	67.06
медиана 	82.14
максимум 	98.42
количество позиций: 38


,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
1563,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 2,5%, бе...","Россия, 900 г",-19%,70.49,56.99,900,63.32
1574,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 900 г",-20%,74.69,59.29,900,65.88
1541,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 1400 г",-14%,110.59,94.89,1400,67.78
1656,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 1,5%...","Россия, 900 мл",NaN,64.79,61.49,900,68.32
1623,"Молоко, сыр, яйцо","Молоко пастеризованное ЭКОНИВА 2,5%, без змж, ...","Россия, 1000 мл",-36%,110.59,69.99,1000,69.99


In [6]:
display(test[test['volume']>600].nlargest(5, columns='Пр. цена'))

,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
1743,"Молоко, сыр, яйцо","Молоко пастеризованное АВИДА 3,2%, без змж, 900мл","Россия, 900 мл",NaN,96.89,91.99,900,102.21
1660,"Молоко, сыр, яйцо","Молоко пастеризованное ПРАВИЛЬНОЕ МОЛОКО 3,2–4...","Россия, 2000 мл",NaN,210.49,199.89,2000,99.94
1642,"Молоко, сыр, яйцо","Молоко пастеризованное ПРОСТОКВАШИНО 3,2%, без...","Россия, 1400 мл",NaN,143.09,135.89,1400,97.06
1853,"Молоко, сыр, яйцо",Молоко пастеризованное ВКУСНОТЕЕВО ультрачисто...,"Россия, 900 г",NaN,91.59,86.99,900,96.66
1670,"Молоко, сыр, яйцо",Молоко пастеризованное ПРОСТОКВАШИНО цельное о...,"Россия, 930 мл",NaN,94.29,89.49,930,96.23


In [7]:
import requests
from bs4 import BeautifulSoup as BS
import lxml
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%B7%D0%B8%D0%BD%D0%B0'

response = requests.get(url)
soup = BS(response.content, 'lxml')

table = soup.find('table', class_='wikitable')
rows = list(table.find_all('tr'))[2:]

sum_list = []
for row in rows[:]:
    out_list = []
    for index, element in enumerate(row.find_all('td')):
        if index > 1:
            per_year = float(element.get_text(strip=True).replace(',', '.'))
            per_month = per_year / 12
            out_list.append(round(per_month, 2))
        else:
            out_list.append(element.get_text(strip=True))
    sum_list.append(out_list)

table_df = pd.DataFrame(sum_list, columns=['name', 'unit', 'works', 'pensioners', 'children'])
table_df

,name,unit,works,pensioners,children
0,Хлебные продукты (хлебимакаронные изделияв пер...,кг,10.54,8.18,6.47
1,Картофель,кг,8.37,6.67,7.34
2,Овощиибахчевые,кг,9.55,8.17,9.38
3,Фруктысвежие,кг,5.00,3.75,9.84
4,Сахарикондитерские изделияв пересчете на сахар,кг,1.98,1.77,1.82
5,Мясопродукты,кг,4.88,4.50,3.67
6,Рыбопродукты,кг,1.54,1.33,1.55
7,Молокои молокопродукты в пересчете на молоко,кг,24.17,21.48,30.06
8,Яйца,штука,17.50,16.67,16.75
9,"Масло растительное,маргарини другиежиры",кг,0.92,0.83,0.42
